Import network and other stuff

In [2]:
import os, joblib, pickle, random, time
from filelock import FileLock
from src.Network import *
from src.dataloader import RNA_Dataset
import ray
from constants import *

Define PATH for results for the playround experiment (with you own data). If you comment this code all the results with real data in 'results' folder will be updated and you want be able to test processing jupyter notes with real result

In [3]:
# PATH_RESULTS = './results_test/'

Define relative paths for datasets

In [4]:
DATA_NAMES = ['1_[res]_[sus, delayedSus]/', '1_[tol]_[sus, delayedSus]/', '[un]_[1]/']

Define names of experiments (multi-domain, domain1, domain2)

In [5]:
EXPERIMENTS = [['spleen', 'liver'], ['spleen'], ['liver']] # i.e. multi-domain, domain1, domain2

Experimental settings are constrained by the following conditions:
N_REPEATS * N_INITS <= NUM_GPUS / GPU_PER_RUN
GPU_PER_RUN * memory of one GPU >= size of network

In [6]:
NUM_GPUS = 2 # number of GPU dedicated to training process
N_REPEATS = 2 # number of repeats
N_INITS = 10 # number of runs in parallel
GPU_PER_RUN = 0.2

Some settings for my particular dataloading flow

In [7]:
spleenDataFile = 'dataSpleen.csv'
liverDataFile = 'dataLiver.csv'
DATA_FILES = {'spleen': spleenDataFile, 'liver': liverDataFile}

Set params of training network

In [8]:
zscore, trainPerc = True, .85
classTrain, AETrain, sparse = True, True, True
latentDim, n_hidden = 128, 1024
nEpochs = 20000
learningRateAE, learningRateD, weightDecay = .0001, .0001, .01
alpha, beta, gamma, lambd = 10.0, 1.0, 0.0001, 0.0001
use_cuda = torch.cuda.is_available()
shuffleTrain = True

Define trainer for specific dataset and experiment

In [9]:
def trainer(DATA_NAME, EXPERIMENT, shuffleTrain):
    
    PATH = PATH_DATA + DATA_NAME
    if len(EXPERIMENT) >1:
        rootDir = PATH_RESULTS + DATA_NAME + 'all/'
    else:
        rootDir = PATH_RESULTS + DATA_NAME + EXPERIMENT[0] + '/'
    os.makedirs(rootDir, exist_ok=True)

    if os.path.exists(rootDir + "weightsList.pickle"):
        try:
            os.remove(rootDir + "weightsList.pickle")
            os.remove(rootDir + "weightsList.pickle.lock")
        except OSError as e:
            print(f"Error: {e.strerror}")
            pass

    ray.init(num_gpus=NUM_GPUS)

    print(f"""
Starting {EXPERIMENT} experiment with the dataset {DATA_NAME} and the following setup:
    N_REPEATS: {N_REPEATS}
    N_INITS: {N_INITS}
    nEpochs: {nEpochs}
          """)

    start = time.time()
    for iter in range(N_REPEATS):

        seed = random.randrange(1, 1000)
        dataset = RNA_Dataset(PATH, DATA_FILES, EXPERIMENT, zscore, trainPerc, seed)
        inputDim = len(dataset.features)
        
        Train, Valid = dict(), dict()
        shuffleValid = False #for visualization shuffleTrain = False
        for _ in EXPERIMENT:
            TrainTemp, ValidTemp = dataset._download(_, shuffleTrain, shuffleValid)
            Train[_] = TrainTemp
            Valid[_] = ValidTemp
        Train = ray.put(Train)
        Valid = ray.put(Valid)

        RemoteNetwork = ray.remote(num_gpus=GPU_PER_RUN)(Network)
        Actors = {}
        for _ in range(N_INITS):
            Actors['Actor: {}'.format(_)] = RemoteNetwork.remote(EXPERIMENT, use_cuda,
                inputDim, latentDim, n_hidden, learningRateAE, learningRateD, weightDecay, alpha, beta, lambd, gamma, 
                classTrain, AETrain, sparse)
        ray.get([value.trainLoop.remote(Train, Valid, nEpochs) for value in Actors.values()])
        weights = ray.get([value.getWeightsSparse.remote() for value in Actors.values()])
        weightsSparse = [_ for _ in weights]

        with FileLock(rootDir + "weightsList.pickle.lock"):
            if os.path.isfile(rootDir + "weightsList.pickle"):
                    listOld = pickle.load( open(rootDir + "weightsList.pickle", "rb" ) )
                    listOld.extend(weightsSparse)
                    pickle.dump( listOld, open( rootDir + "weightsList.pickle", "wb" ) )
            else:
                pickle.dump(weightsSparse, open(rootDir + "weightsList.pickle", "wb" ) )
    ray.shutdown()
    print(f"Training for the experiment {EXPERIMENT} with the  dataset {DATA_NAME} took {time.time()-start} seconds")

Run all both multi-domain and two one domain trainings for one dataset '1_[res]_[sus, delayedSus]', i.e. 'resistant' versus 'susceptible'

In [10]:
DATA_NAME = '1_[res]_[sus, delayedSus]/'
for EXPERIMENT in EXPERIMENTS:
    trainer(DATA_NAME, EXPERIMENT, shuffleTrain)

# # to run all datasets:
# for DATA_NAME in DATA_NAMES:
#     for EXPERIMENT in EXPERIMENTS:
#         trainer(DATA_NAME, EXPERIMENT, shuffleTrain)

2024-04-17 03:32:10,201	INFO worker.py:1518 -- Started a local Ray instance.



Starting ['spleen', 'liver'] experiment with the dataset 1_[res]_[sus, delayedSus]/ and the following setup:
    N_REPEATS: 2
    N_INITS: 10
    nEpochs: 20000
          
Training for the experiment ['spleen', 'liver'] with the  dataset 1_[res]_[sus, delayedSus]/ took 4191.774039983749 seconds


2024-04-17 04:42:04,608	INFO worker.py:1518 -- Started a local Ray instance.



Starting ['spleen'] experiment with the dataset 1_[res]_[sus, delayedSus]/ and the following setup:
    N_REPEATS: 2
    N_INITS: 10
    nEpochs: 20000
          
Training for the experiment ['spleen'] with the  dataset 1_[res]_[sus, delayedSus]/ took 2398.7983787059784 seconds


2024-04-17 05:22:07,267	INFO worker.py:1518 -- Started a local Ray instance.



Starting ['liver'] experiment with the dataset 1_[res]_[sus, delayedSus]/ and the following setup:
    N_REPEATS: 2
    N_INITS: 10
    nEpochs: 20000
          
Training for the experiment ['liver'] with the  dataset 1_[res]_[sus, delayedSus]/ took 1927.613466501236 seconds
